# **Temporal Visualizations of Flight Delays in the US (2019-2023)**

**Name:** Jayanth Anala \\
**NetID:** ja4874

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# Importing from dataset file
Below is a helper function that imports raw data from the url provided, and returns the final dataframe.

In [ ]:
def import_df(filename):
    df = pd.read_csv(filename)
    return df

delay_df = import_df("/content/drive/MyDrive/IV Datasets/flights_sample_3m.csv")
airport_df = import_df("/content/drive/MyDrive/IV Datasets/airport_data.csv")

In [ ]:
delay_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 32 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   FL_DATE                  object 
 1   AIRLINE                  object 
 2   AIRLINE_DOT              object 
 3   AIRLINE_CODE             object 
 4   DOT_CODE                 int64  
 5   FL_NUMBER                int64  
 6   ORIGIN                   object 
 7   ORIGIN_CITY              object 
 8   DEST                     object 
 9   DEST_CITY                object 
 10  CRS_DEP_TIME             int64  
 11  DEP_TIME                 float64
 12  DEP_DELAY                float64
 13  TAXI_OUT                 float64
 14  WHEELS_OFF               float64
 15  WHEELS_ON                float64
 16  TAXI_IN                  float64
 17  CRS_ARR_TIME             int64  
 18  ARR_TIME                 float64
 19  ARR_DELAY                float64
 20  CANCELLED                float64
 21  CANCELLA

In [ ]:
airport_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   code       380 non-null    object 
 1   Latitude   380 non-null    float64
 2   Longitude  380 non-null    float64
dtypes: float64(2), object(1)
memory usage: 9.0+ KB


# Understanding the Datasets

In [ ]:
## Pre Removal - Getting info
delay_df.info()
display(delay_df)

start_nan_count = delay_df['DELAY_DUE_WEATHER'].isna().sum()
end_nan_count = delay_df['FL_DATE'].isna().sum()
print(start_nan_count, end_nan_count)



## Removal - Removing NaN values from start and end stations
# delay_df.dropna(subset=['start_station_id','end_station_id'],inplace = True)
# delay_df['start_station_id'] = delay_df['start_station_id'].astype(int)
# delay_df['end_station_id'] = delay_df['end_station_id'].astype(int)
# trips_pr_df = delay_df

## Post Removal - getting info
# trips_pr_df.info()
# start_nan_count = trips_pr_df['start_station_id'].isna().sum()
# end_nan_count = trips_pr_df['end_station_id'].isna().sum()
# print(start_nan_count,end_nan_count)

# **Dataset Cleaning**

First, I'm removing the columns that are insignificant in this study like DOT code, Cancellation code, Taxi In/Out codes, Wheels On/Off Times

In [ ]:
delay_df.drop(['AIRLINE_DOT', 'AIRLINE_CODE','DOT_CODE', 'FL_NUMBER', 'CRS_DEP_TIME','TAXI_OUT','WHEELS_OFF','WHEELS_ON','TAXI_IN','CRS_ARR_TIME','CANCELLATION_CODE','CRS_ELAPSED_TIME'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 20 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   FL_DATE                  object 
 1   AIRLINE                  object 
 2   ORIGIN                   object 
 3   ORIGIN_CITY              object 
 4   DEST                     object 
 5   DEST_CITY                object 
 6   DEP_TIME                 float64
 7   DEP_DELAY                float64
 8   ARR_TIME                 float64
 9   ARR_DELAY                float64
 10  CANCELLED                float64
 11  DIVERTED                 float64
 12  ELAPSED_TIME             float64
 13  AIR_TIME                 float64
 14  DISTANCE                 float64
 15  DELAY_DUE_CARRIER        float64
 16  DELAY_DUE_WEATHER        float64
 17  DELAY_DUE_NAS            float64
 18  DELAY_DUE_SECURITY       float64
 19  DELAY_DUE_LATE_AIRCRAFT  float64
dtypes: float64(14), object(6)
memory usage: 457.8+

In [ ]:
delay_df['AVG_DELAY_TIME'] = delay_df[['DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS', 'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT']].mean(axis=1)

**Adding Total delay times column based on all types of delays**

In [ ]:
delay_df['total_delay_time'] = delay_df[['DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS', 'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT']].sum(axis=1)

In [ ]:
# Categorizing the type of delay based on the arrival times

def categorize_delay(delay,cancel):
    if delay < 0:
        return 'Early'
    elif delay == 0:
        return 'On-time'
    elif delay > 0:
        return 'Delayed'
    elif cancel == 1:
        return 'Cancelled'
    else:
        return 'Diverted'

delay_df['ARR_Type'] = delay_df.apply(lambda x: categorize_delay(x['ARR_DELAY'], x['CANCELLED']), axis=1)

# **Datasets Merging**

In [ ]:
# Merging the two datasets based on ORIGIN airport code
merged_df1 = pd.merge(delay_df, airport_df, left_on="ORIGIN", right_on="code", how="left")
merged_df1.drop(columns=["code"], inplace=True)
merged_df1.rename(columns={"Latitude": "origin_lat", "Longitude": "origin_lon"}, inplace=True)

# Merging the two datasets based on DEST airport code
merged_df2 = pd.merge(delay_df, airport_df, left_on="DEST", right_on="code", how="left")
merged_df2.drop(columns=["code"], inplace=True)
merged_df2.rename(columns={"Latitude": "destination_lat", "Longitude": "destination_lon"}, inplace=True)

# Combine the two merged dfs
delay_df = pd.concat([merged_df1, merged_df2[['destination_lat', 'destination_lon']]], axis=1)

# Splitting the State and City into two columns
delay_df[['ORIGIN_CITY', 'ORIGIN_STATE']] = delay_df['ORIGIN_CITY'].str.split(', ', expand=True)
delay_df[['DEST_CITY', 'DEST_STATE']] = delay_df['DEST_CITY'].str.split(', ', expand=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 27 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   FL_DATE                  object 
 1   AIRLINE                  object 
 2   ORIGIN                   object 
 3   ORIGIN_CITY              object 
 4   DEST                     object 
 5   DEST_CITY                object 
 6   DEP_TIME                 float64
 7   DEP_DELAY                float64
 8   ARR_TIME                 float64
 9   ARR_DELAY                float64
 10  CANCELLED                float64
 11  DIVERTED                 float64
 12  ELAPSED_TIME             float64
 13  AIR_TIME                 float64
 14  DISTANCE                 float64
 15  DELAY_DUE_CARRIER        float64
 16  DELAY_DUE_WEATHER        float64
 17  DELAY_DUE_NAS            float64
 18  DELAY_DUE_SECURITY       float64
 19  DELAY_DUE_LATE_AIRCRAFT  float64
 20  total_delay_time         float64
 21  origin_l

,FL_DATE,AIRLINE,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,...,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT,total_delay_time,origin_lat,origin_lon,destination_lat,destination_lon,ORIGIN_STATE,DEST_STATE
0,2019-01-09,United Air Lines Inc.,FLL,Fort Lauderdale,EWR,Newark,1151.0,-4.0,1447.0,-14.0,...,NaN,NaN,NaN,0.0,26.0726,-80.1527,40.6925,-74.1687,FL,NJ
1,2022-11-19,Delta Air Lines Inc.,MSP,Minneapolis,SEA,Seattle,2114.0,-6.0,2310.0,-5.0,...,NaN,NaN,NaN,0.0,44.8820,-93.2218,47.4490,-122.3090,MN,WA
2,2022-07-22,United Air Lines Inc.,DEN,Denver,MSP,Minneapolis,1000.0,6.0,1252.0,0.0,...,NaN,NaN,NaN,0.0,39.8617,-104.6730,44.8820,-93.2218,CO,MN
3,2023-03-06,Delta Air Lines Inc.,MSP,Minneapolis,SFO,San Francisco,1608.0,-1.0,1853.0,24.0,...,24.0,0.0,0.0,24.0,44.8820,-93.2218,37.6190,-122.3750,MN,CA
4,2020-02-23,Spirit Air Lines,MCO,Orlando,DFW,Dallas/Fort Worth,1838.0,-2.0,2040.0,-1.0,...,NaN,NaN,NaN,0.0,28.4294,-81.3090,32.8968,-97.0380,FL,TX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2022-11-13,American Airlines Inc.,JAX,Jacksonville,CLT,Charlotte,1740.0,-2.0,1851.0,-16.0,...,NaN,NaN,NaN,0.0,30.4941,-81.6879,35.2140,-80.9431,FL,NC
2999996,2022-11-02,American Airlines Inc.,ORD,Chicago,AUS,Austin,1254.0,-6.0,1519.0,-37.0,...,NaN,NaN,NaN,0.0,41.9786,-87.9048,30.1945,-97.6699,IL,TX
2999997,2022-09-11,Delta Air Lines Inc.,HSV,Huntsville,ATL,Atlanta,615.0,41.0,805.0,36.0,...,0.0,0.0,0.0,36.0,34.6372,-86.7751,33.6367,-84.4281,AL,GA
2999998,2019-11-13,Republic Airline,BOS,Boston,LGA,New York,1555.0,-5.0,1712.0,-16.0,...,NaN,NaN,NaN,0.0,42.3643,-71.0052,40.7772,-73.8726,MA,NY


In [ ]:
display(delay_df)

delay_df.to_csv("flight_delay_df.csv", index=False)

,FL_DATE,AIRLINE,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,...,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT,total_delay_time,origin_lat,origin_lon,destination_lat,destination_lon,ORIGIN_STATE,DEST_STATE,ARR_Type
0,2019-01-09,United Air Lines Inc.,FLL,Fort Lauderdale,EWR,Newark,1151.0,-4.0,1447.0,-14.0,...,NaN,NaN,0.0,26.0726,-80.1527,40.6925,-74.1687,FL,NJ,Early
1,2022-11-19,Delta Air Lines Inc.,MSP,Minneapolis,SEA,Seattle,2114.0,-6.0,2310.0,-5.0,...,NaN,NaN,0.0,44.8820,-93.2218,47.4490,-122.3090,MN,WA,Early
2,2022-07-22,United Air Lines Inc.,DEN,Denver,MSP,Minneapolis,1000.0,6.0,1252.0,0.0,...,NaN,NaN,0.0,39.8617,-104.6730,44.8820,-93.2218,CO,MN,On-time
3,2023-03-06,Delta Air Lines Inc.,MSP,Minneapolis,SFO,San Francisco,1608.0,-1.0,1853.0,24.0,...,0.0,0.0,24.0,44.8820,-93.2218,37.6190,-122.3750,MN,CA,Delayed
4,2020-02-23,Spirit Air Lines,MCO,Orlando,DFW,Dallas/Fort Worth,1838.0,-2.0,2040.0,-1.0,...,NaN,NaN,0.0,28.4294,-81.3090,32.8968,-97.0380,FL,TX,Early
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2022-11-13,American Airlines Inc.,JAX,Jacksonville,CLT,Charlotte,1740.0,-2.0,1851.0,-16.0,...,NaN,NaN,0.0,30.4941,-81.6879,35.2140,-80.9431,FL,NC,Early
2999996,2022-11-02,American Airlines Inc.,ORD,Chicago,AUS,Austin,1254.0,-6.0,1519.0,-37.0,...,NaN,NaN,0.0,41.9786,-87.9048,30.1945,-97.6699,IL,TX,Early
2999997,2022-09-11,Delta Air Lines Inc.,HSV,Huntsville,ATL,Atlanta,615.0,41.0,805.0,36.0,...,0.0,0.0,36.0,34.6372,-86.7751,33.6367,-84.4281,AL,GA,Delayed
2999998,2019-11-13,Republic Airline,BOS,Boston,LGA,New York,1555.0,-5.0,1712.0,-16.0,...,NaN,NaN,0.0,42.3643,-71.0052,40.7772,-73.8726,MA,NY,Early
